Tarea 4: Predicción del score de la review
¿Qué tan difícil es predecir el score de la review? ¿Cuál es el modelo más sencillo que
puede construirse con resultados aceptables? ¿Cuál es la menor cantidad de datos que
puede usarse para tener datos aceptables?

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [3]:
!pip install category_encoders

In [63]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score,auc
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,OneHotEncoder
from sklearn.linear_model import LogisticRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import category_encoders.utils as util
import seaborn as sns

In [5]:
reviews = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TP4/silkroad_reviews.csv')

In [6]:
reviews

,item_id,date_created,comment,rating
0,250-grams-ghb-powder,2013-11-28,"fast shipping, great product",5.0
1,10g-washed-fishscale-cocaine,2013-11-28,Amazing product! Perfect stealth. Will def...,5.0
2,roche-valium-10mg,2013-11-28,"good vendor, posted express when paid for regu...",5.0
3,10g-washed-fishscale-cocaine,2013-11-28,great stuff,5.0
4,influence-the-psychology-of-persuasion,2013-11-28,Item received as advertised. Quick response. G...,5.0
...,...,...,...,...
314204,1g-pure-coke-listing,2014-11-05,"Good gear but not the cleanest IMHO, each to t...",0.0
314205,oxycontin-80mg-oc-oxycodone-hydrochloride-1-bo...,2014-11-05,this man is god...love u dok,5.0
314206,7g-stinky-bud,2014-11-05,Great product - stealth not amazing,5.0
314207,imported-dutch-mdma-uncut-28-grams-one-ounce,2014-11-05,A+,5.0


# Baseline

## Encodiado de los Data Frame

In [38]:
reviews = reviews.loc[:][reviews.rating <=5]

In [39]:
limite = reviews.index.size*0.9

In [40]:
top = reviews.index > limite #validation

In [41]:
down = reviews.index <= limite #validation

In [42]:
def encoderCountVectorizer(column,dataFrame,*form):
  if(form[0]=='fit_transform'):
    le[column] = TfidfVectorizer(max_features=500)
    feature_encoder = le[column].fit_transform(dataFrame[column].astype(str))
  else:
    feature_encoder = le[column].transform(dataFrame[column].astype(str))
  del(dataFrame[column])
  return dataFrame.join(pd.DataFrame(feature_encoder.todense()).add_prefix(column+'_'))

In [43]:
le = {}
reviews = encoderCountVectorizer('comment',reviews,'fit_transform')

In [44]:
validation = reviews[top]

In [45]:
train = reviews[down]

In [47]:
def encoderOrdinal(dataFrame,*form):
  for column in dataFrame.columns:
    if dataFrame[column].dtype==object:
        if(form[0]=='fit_transform'):
          le[column] = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=np.nan)
          dataFrame[column] = le[column].fit_transform(dataFrame[column].astype(str).values.reshape(-1,1))
        else:
          dataFrame[column] = le[column].transform(dataFrame[column].astype(str).values.reshape(-1,1))

In [48]:
encoderOrdinal(train,'fit_transform')
encoderOrdinal(validation,'transform')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

## Imputacion de Nulos

In [49]:
train.fillna(-1,inplace=True)
validation.fillna(-1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


## Entrenamiento del modelo

In [50]:
label_train = train.rating
label_validation = validation.rating

In [51]:
train

,item_id,date_created,rating,comment_0,comment_1,comment_2,comment_3,comment_4,comment_5,comment_6,comment_7,comment_8,comment_9,comment_10,comment_11,comment_12,comment_13,comment_14,comment_15,comment_16,comment_17,comment_18,comment_19,comment_20,comment_21,comment_22,comment_23,comment_24,comment_25,comment_26,comment_27,comment_28,comment_29,comment_30,comment_31,comment_32,comment_33,comment_34,comment_35,comment_36,...,comment_460,comment_461,comment_462,comment_463,comment_464,comment_465,comment_466,comment_467,comment_468,comment_469,comment_470,comment_471,comment_472,comment_473,comment_474,comment_475,comment_476,comment_477,comment_478,comment_479,comment_480,comment_481,comment_482,comment_483,comment_484,comment_485,comment_486,comment_487,comment_488,comment_489,comment_490,comment_491,comment_492,comment_493,comment_494,comment_495,comment_496,comment_497,comment_498,comment_499
0,3165.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1686.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.323517,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.389132,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.304138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10147.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.334046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.601422,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1686.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8426.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.461026,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24348,0.0,0.0,...,0.0,0.237320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282781,1218.0,311.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.473393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
282782,1221.0,311.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
282783,1123.0,311.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [52]:
del(train['rating'])
del(validation['rating'])

In [53]:
from sklearn.ensemble import RandomForestClassifier

In [54]:
search = RandomForestClassifier(random_state=0).fit(train.values,label_train.values)

## Score con las metricas utilizada

In [68]:
roc_auc_score(label_validation,search.predict_proba(validation.values)[:,1],multi_class='ovr')

0.49679327790539196

# Red Neuronal

In [57]:
#ax = sns.histplot(data=label_validation.to_frame())
#sns.set(rc = {"figure.dpi":200,})
#figure = ax.get_figure()    
#figure.savefig('/content/drive/MyDrive/Colab Notebooks/TP4/LabelsVal.png')

In [58]:
#logistic = xgb.XGBClassifier()
#distributions = dict(subsample=[1,0.8,0.7,0.5,0.4,0.2],booster=['gbtree','gblinear','dart'],n_estimators=list(range(50,200,50)),learning_rate=[0.35,0.3,0.25,0.28,0.2,0.15],max_depth=list(range(5,30,5)),tree_method=['auto','exact','hist'])
#clf = RandomizedSearchCV(logistic, distributions, random_state=0,n_iter=3,verbose=100,cv=3,scoring='roc_auc')
#search = clf.fit(train.values,label_train.values)
#search.best_params_